In [24]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib as mpl
from matplotlib import font_manager, rc
%matplotlib inline

# 글꼴 경로 지정
font_path = "c:/Windows/Fonts/malgun.ttf"  # 윈도우에 설치된 맑은 고딕 폰트 경로

# 폰트 이름 얻어오기
font_name = font_manager.FontProperties(fname=font_path).get_name()

# matplotlib의 rc(run command) 기능을 이용하여 글꼴 설정
mpl.rc('font', family=font_name)

# 유니코드에서  음수 부호 설정
mpl.rc('axes', unicode_minus=False)

df_guest_prefer = pd.read_csv('../../../../../datasets/paris_prefer.csv')
df_non_guest_prefer= pd.read_csv('../../../../../datasets/paris_non_prefer.csv')

# paris metro station

In [56]:
# CSV 파일 로드
df_metro = pd.read_csv('../../../../../datasets/paris_stations.csv', delimiter=';')

# 'Coordonnées géographiques' 열을 공백으로 분리하여 새로운 열로 추가
# 데이터에서 쉼표도 제거 필요
df_metro[['위도', '경도']] = df_metro['Coordonnées géographiques'].str.replace(',', '').str.split(expand=True)

# 위도와 경도 데이터 타입을 float으로 변환
df_metro['위도'] = df_metro['위도'].astype(float)
df_metro['경도'] = df_metro['경도'].astype(float)

# 필요없는 'Coordonnées géographiques' 열 제거
df_metro.drop('Coordonnées géographiques', axis=1, inplace=True)

# 결과 확인
print(df_metro.head())


  Identifiant station                    Nom de la station  \
0               17044      Porte de Saint-Ouen - Bessières   
1               20143               Ramponeau - Belleville   
2               16138             Saint-Cloud - Hippodrome   
3               17008              Porte Pouchet-Bessières   
4               10107  Gare du Nord - Hôpital Lariboisière   

   Capacité de la station         위도        경도  
0                      42  48.897922  2.328515  
1                      44  48.871031  2.379057  
2                      30  48.857721  2.254911  
3                      23  48.897877  2.323063  
4                      49  48.881949  2.352339  


In [57]:
df_metro = df_metro.rename(columns= {
    'Identifiant station':'역_id',
    'Nom de la station' : '역이름',
    'Capacité de la station' : '역수용인원'
})
df_metro= df_metro[df_metro['역수용인원']!=0]

In [69]:
# 역 이름을 '-'로 분할하고 첫 번째 부분을 새로운 열에 저장
df_metro['역 이름 앞부분'] = df_metro['역이름'].apply(lambda x: x.split(' - ')[0])


# '역 이름 앞부분'으로 그룹화하고, 각 그룹의 위도와 경도 평균을 계산
station_groups = df_metro.groupby('역 이름 앞부분').agg({
    '위도': 'mean',
    '경도': 'mean'
}).reset_index()

# 새로운 DataFrame 생성
df_station = station_groups.rename(columns={
    '역 이름 앞부분': '역이름'
})


In [68]:
len(df_metro['역 이름 앞부분'].unique())

1176

In [59]:
len(df_metro['역 이름 앞부분'].unique())

1176

In [26]:
pip install shapely


Note: you may need to restart the kernel to use updated packages.


In [28]:
pip install folium


Note: you may need to restart the kernel to use updated packages.


In [27]:
pip install geopy

Note: you may need to restart the kernel to use updated packages.


In [71]:
import folium
from folium.plugins import MarkerCluster
from folium import FeatureGroup, LayerControl
from folium.features import GeoJson
from shapely.geometry import shape, Point
import json

# 파리 중심부의 위도와 경도
paris_latitude = 48.8566
paris_longitude = 2.3522

# 파리 지도 생성
map_paris = folium.Map(location=[paris_latitude, paris_longitude], zoom_start=12)
# 랜드마크 위치 데이터
landmarks = {
    "에펠탑": (48.8584, 2.2945),
    "루브르 박물관": (48.8606, 2.3376),
    "노트르담 대성당": (48.8529, 2.3500),
    "개선문": (48.8738, 2.2950),
    "가르니에 궁전": (48.8719, 2.3316),
    "사크레쾨르 대성당": (48.8867, 2.3431),
    "생트 샤펠": (48.8554, 2.3450),
    "알렉산드르 3세 다리": (48.8639, 2.3136),
    "마레 지구": (48.8575, 2.3588),
    "몽마르트": (48.8867, 2.3431),
    "팡테옹": (48.8463, 2.3460),
    "룩셈부르크 정원": (48.8462, 2.3371),
    "오르세 미술관": (48.8600, 2.3257),
    "샹젤리제 거리": (48.8696, 2.3079),
    "레 장발리드": (48.8565, 2.3126),
    "콩코르드 광장": (48.8656, 2.3211),
    "퐁피두 센터": (48.8606, 2.3522),
    "라 데팡스": (48.8900, 2.2400),
    "갤러리 라파예트": (48.8738, 2.3320),
    "그레뱅 박물관": (48.8718, 2.3422)
}
# 파리 경계 데이터 로드
with open('./파리_data/paris.geojson', 'r') as f:
    paris_boundary = json.load(f)

# shapely geometry 객체 생성
paris_area = shape(paris_boundary['features'][0]['geometry'])

# 마커 클러스터 생성
marker_cluster = MarkerCluster().add_to(map_paris)

# 마커 추가 조건 검사
def add_marker_if_inside(lat, lng, popup, icon_color):
    point = Point(lng, lat)
    if paris_area.contains(point):
        folium.Marker(
            location=[lat, lng],
            icon=folium.Icon(color=icon_color, icon='info-sign'),
            popup=popup
        ).add_to(marker_cluster)

# 사용자 데이터를 반복하면서 마커 추가
for idx, row in df_guest_prefer.iterrows():
    add_marker_if_inside(row['위도'], row['경도'], 'Guest Prefer', 'green')

for idx, row in df_non_guest_prefer.iterrows():
    add_marker_if_inside(row['위도'], row['경도'], 'Non-Guest Prefer', 'red')
# 각 랜드마크에 대한 마커 추가
for landmark, (lat, lng) in landmarks.items():
    folium.Marker(
        location=[lat, lng],
        icon=folium.Icon(icon='star', color='blue'),
        popup=landmark
    ).add_to(map_paris)


# df_guest_prefer의 위치 표시 (클러스터에 추가)
for idx, row in df_guest_prefer.iterrows():
    folium.Marker(
        location=[row['위도'], row['경도']],
        icon=folium.Icon(color='green', icon='info-sign'),
        popup='Guest Prefer'
    ).add_to(marker_cluster)

# df_non_guest_prefer의 위치 표시 (클러스터에 추가)
for idx, row in df_non_guest_prefer.iterrows():
    folium.Marker(
        location=[row['위도'], row['경도']],
        icon=folium.Icon(color='red', icon='info-sign'),
        popup='Non-Guest Prefer'
    ).add_to(marker_cluster)

# 지하철, 기차, 트램 데이터를 각각의 레이어로 추가
geojson_data_train = './파리_data/paris_train.geojson'
geojson_data_tram = './파리_data/paris_tram.geojson'

paris_boundary_geojson = './파리_data/paris.geojson'

# 파리 경계 레이어 추가
GeoJson(
    paris_boundary_geojson,
    name='Paris Boundary',
    style_function=lambda x: {
        'fillColor': 'blue',
        'color': 'blue',
        'weight': 2,
        'fillOpacity': 0.1
    }
).add_to(map_paris)

# 지하철 레이어# 지하철 레이어를 새롭게 생성하고 기존 레이어 대체
subway_layer = FeatureGroup(name='Subway Stations', show=False)  # show=False로 설정하여 초기에는 표시되지 않게 합니다.

# df_station 데이터를 이용해 지하철역 위치에 마커 추가
for idx, row in df_station.iterrows():
    folium.CircleMarker(
        location=[row['위도'], row['경도']],
        radius=5,  # 마커의 반경
        color='blue',  # 마커의 색상
        fill=True,
        fill_color='blue',
        fill_opacity=0.7,
        popup=f"Station"  # 팝업에 표시할 텍스트
    ).add_to(subway_layer)


# 지하철 레이어를 지도에 추가
subway_layer.add_to(map_paris)

# 기차 레이어
train_layer = FeatureGroup(name='Train Stations')
GeoJson(geojson_data_train, name='Train Stations').add_to(train_layer)
train_layer.add_to(map_paris)

# 트램 레이어
tram_layer = FeatureGroup(name='Tram Stops')
GeoJson(geojson_data_tram, name='Tram Stops').add_to(tram_layer)
tram_layer.add_to(map_paris)

# 레이어 컨트롤 추가
LayerControl().add_to(map_paris)

# 지도를 HTML 파일로 저장
map_paris.save('paris_transport_map.html')

In [8]:
from shapely.geometry import shape, Point
import json

# GeoJSON 파일 불러오기
with open('./파리_data/paris_subway.geojson', 'r') as f:
    subway_data = json.load(f)

with open('./파리_data/paris_train.geojson', 'r') as f:
    train_data = json.load(f)

with open('./파리_data/paris_tram.geojson', 'r') as f:
    tram_data = json.load(f)

# 숙소 데이터 DataFrame에서 위도와 경도로 포인트 생성
df_guest_prefer['point'] = df_guest_prefer.apply(lambda row: Point(row['경도'], row['위도']), axis=1)
df_non_guest_prefer['point'] = df_non_guest_prefer.apply(lambda row: Point(row['경도'], row['위도']), axis=1)

# 각 정거장 위치에 대해 500미터 이내 숙소 개수 계산
def count_nearby_accommodations(geojson_data, df, radius=500):
    count = 0
    for feature in geojson_data['features']:
        station_point = shape(feature['geometry'])
        # 각 숙소와 정거장 간의 거리 계산
        nearby_acco mmodations = df[df['point'].apply(lambda x: x.distance(station_point) * (111.32 * 1000)) <= radius]
        count += len(nearby_accommodations)
    return count

# 지하철, 기차, 트램 정거장별로 숙소 개수 계산
subway_nearby_guest_prefer = count_nearby_accommodations(subway_data, df_guest_prefer)
subway_nearby_non_guest_prefer = count_nearby_accommodations(subway_data, df_non_guest_prefer)

train_nearby_guest_prefer = count_nearby_accommodations(train_data, df_guest_prefer)
train_nearby_non_guest_prefer = count_nearby_accommodations(train_data, df_non_guest_prefer)

tram_nearby_guest_prefer = count_nearby_accommodations(tram_data, df_guest_prefer)
tram_nearby_non_guest_prefer = count_nearby_accommodations(tram_data, df_non_guest_prefer)

# 결과 출력
print("지하철 근처 숙소 개수 (게스트 선호):", subway_nearby_guest_prefer)
print("지하철 근처 숙소 개수 (비게스트 선호):", subway_nearby_non_guest_prefer)
print("기차 근처 숙소 개수 (게스트 선호):", train_nearby_guest_prefer)
print("기차 근처 숙소 개수 (비게스트 선호):", train_nearby_non_guest_prefer)
print("트램 근처 숙소 개수 (게스트 선호):", tram_nearby_guest_prefer)
print("트램 근처 숙소 개수 (비게스트 선호):", tram_nearby_non_guest_prefer)


지하철 근처 숙소 개수 (게스트 선호): 9061
지하철 근처 숙소 개수 (비게스트 선호): 9415
기차 근처 숙소 개수 (게스트 선호): 5
기차 근처 숙소 개수 (비게스트 선호): 14
트램 근처 숙소 개수 (게스트 선호): 1014
트램 근처 숙소 개수 (비게스트 선호): 1081


In [73]:
from geopy.distance import geodesic

# 특정 랜드마크와의 거리 계산 함수 정의
def count_accommodations_within_distance(df, landmark_coords, max_distance_km):
    count = 0
    for idx, row in df.iterrows():
        # 숙소 위치
        accommodation_location = (row['위도'], row['경도'])
        # 랜드마크와 숙소 간의 거리 계산
        distance = geodesic(landmark_coords, accommodation_location).km
        # 특정 거리 이내에 있으면 카운트
        if distance <= max_distance_km:
            count += 1
    return count

# 각 랜드마크와의 거리 계산
landmark_distances = {}
for landmark, coords in landmarks.items():
    count_prefer = count_accommodations_within_distance(df_guest_prefer, coords, 1)  # 1km 이내
    count_non_prefer = count_accommodations_within_distance(df_non_guest_prefer, coords, 1)  # 1km 이내
    landmark_distances[landmark] = {'Guest Prefer': count_prefer, 'Non-Guest Prefer': count_non_prefer}

# 결과 출력
for landmark, counts in landmark_distances.items():
    print(f"{landmark}:  1km 내 게스트 선호: {counts['Guest Prefer']}, 게스트 불호: {counts['Non-Guest Prefer']}")


에펠탑:  1km 내 게스트 선호: 167, 게스트 불호: 160
루브르 박물관:  1km 내 게스트 선호: 318, 게스트 불호: 339
노트르담 대성당:  1km 내 게스트 선호: 382, 게스트 불호: 352
개선문:  1km 내 게스트 선호: 176, 게스트 불호: 207
가르니에 궁전:  1km 내 게스트 선호: 230, 게스트 불호: 303
사크레쾨르 대성당:  1km 내 게스트 선호: 529, 게스트 불호: 397
생트 샤펠:  1km 내 게스트 선호: 387, 게스트 불호: 376
알렉산드르 3세 다리:  1km 내 게스트 선호: 87, 게스트 불호: 120
마레 지구:  1km 내 게스트 선호: 524, 게스트 불호: 490
몽마르트:  1km 내 게스트 선호: 529, 게스트 불호: 397
팡테옹:  1km 내 게스트 선호: 228, 게스트 불호: 221
룩셈부르크 정원:  1km 내 게스트 선호: 224, 게스트 불호: 219
오르세 미술관:  1km 내 게스트 선호: 114, 게스트 불호: 88
샹젤리제 거리:  1km 내 게스트 선호: 97, 게스트 불호: 153
레 장발리드:  1km 내 게스트 선호: 102, 게스트 불호: 90
콩코르드 광장:  1km 내 게스트 선호: 111, 게스트 불호: 88
퐁피두 센터:  1km 내 게스트 선호: 597, 게스트 불호: 694
라 데팡스:  1km 내 게스트 선호: 0, 게스트 불호: 0
갤러리 라파예트:  1km 내 게스트 선호: 254, 게스트 불호: 347
그레뱅 박물관:  1km 내 게스트 선호: 428, 게스트 불호: 626


In [74]:
from geopy.distance import geodesic

def count_accommodations_within_distance(station_coords, accommodations_df, max_distance=0.5):
    """주어진 역 위치에서 max_distance(km) 이내에 있는 숙소의 수를 반환합니다."""
    count = 0
    for idx, row in accommodations_df.iterrows():
        accommodation_coords = (row['위도'], row['경도'])
        distance = geodesic(station_coords, accommodation_coords).km
        if distance <= max_distance:
            count += 1
    return count

# 각 역에 대해 숙소 수를 계산
results = []
for idx, row in df_station.iterrows():
    station_coords = (row['위도'], row['경도'])
    nearby_guest_prefer = count_accommodations_within_distance(station_coords, df_guest_prefer)
    nearby_non_guest_prefer = count_accommodations_within_distance(station_coords, df_non_guest_prefer)
    results.append({
        '역이름': row['역이름'],
        '게스트 선호 숙소 수': nearby_guest_prefer,
        '비게스트 선호 숙소 수': nearby_non_guest_prefer
    })

# 결과 DataFrame 생성
df_results = pd.DataFrame(results)
print(df_results)

                   역이름  게스트 선호 숙소 수  비게스트 선호 숙소 수
0     11 Novembre 1918            0             0
1         18 juin 1940            0             0
2           8 Mai 1945            0             0
3            Abbeville          103           106
4          Abbé Carton           30            38
...                ...          ...           ...
1171             Wurtz           31            13
1172    Youri Gagarine            0             0
1173       Yves Farges            0             0
1174  Édouard Vaillant            0             0
1175   Île de la Jatte            0             0

[1176 rows x 3 columns]


In [75]:
df_results['게스트 선호 숙소 수'].sum()

42745

In [76]:
df_results.to_csv('./check1.csv',index=False)

In [9]:
len(df_guest_prefer)

4716

  Identifiant station                    Nom de la station  \
0               17044      Porte de Saint-Ouen - Bessières   
1               20143               Ramponeau - Belleville   
2               16138             Saint-Cloud - Hippodrome   
3               17008              Porte Pouchet-Bessières   
4               10107  Gare du Nord - Hôpital Lariboisière   

   Capacité de la station         위도        경도  
0                      42  48.897922  2.328515  
1                      44  48.871031  2.379057  
2                      30  48.857721  2.254911  
3                      23  48.897877  2.323063  
4                      49  48.881949  2.352339  


In [11]:
a1

,Identifiant station;Nom de la station;Capacité de la station;Coordonnées géographiques
17044;Porte de Saint-Ouen - Bessières;42;48.89792240854517,2.328515
20143;Ramponeau - Belleville;44;48.871031137596,2.379057
16138;Saint-Cloud - Hippodrome;30;48.85772148887421,2.254911
17008;Porte Pouchet-Bessières;23;48.8978765,2.323063
10107;Gare du Nord - Hôpital Lariboisière;49;48.881949,2.352339
...,...
15058;Place du Moulin de Javel;45;48.84021670922,2.271606
11020;Belfort - Voltaire;39;48.8563358,2.382990
11012;Montreuil - Voltaire;34;48.850507011842,2.393076
4010;Saint-Antoine Sévigné;26;48.8550222,2.361232


In [21]:
df_metro.head()

,Identifiant station,Nom de la station,Capacité de la station,위도,경도
0,17044,Porte de Saint-Ouen - Bessières,42,48.897922,2.328515
1,20143,Ramponeau - Belleville,44,48.871031,2.379057
2,16138,Saint-Cloud - Hippodrome,30,48.857721,2.254911
3,17008,Porte Pouchet-Bessières,23,48.897877,2.323063
4,10107,Gare du Nord - Hôpital Lariboisière,49,48.881949,2.352339


In [23]:
import folium
from folium.plugins import MarkerCluster
from folium import FeatureGroup, LayerControl
from folium.features import GeoJson
from shapely.geometry import shape, Point
import json

# 파리 중심부의 위도와 경도
paris_latitude = 48.8566
paris_longitude = 2.3522

# 파리 지도 생성
map_paris = folium.Map(location=[paris_latitude, paris_longitude], zoom_start=12)
# 랜드마크 위치 데이터
landmarks = {
    "에펠탑": (48.8584, 2.2945),
    "루브르 박물관": (48.8606, 2.3376),
    "노트르담 대성당": (48.8529, 2.3500),
    "개선문": (48.8738, 2.2950),
    "가르니에 궁전": (48.8719, 2.3316),
    "사크레쾨르 대성당": (48.8867, 2.3431),
    "생트 샤펠": (48.8554, 2.3450),
    "알렉산드르 3세 다리": (48.8639, 2.3136),
    "마레 지구": (48.8575, 2.3588),
    "몽마르트": (48.8867, 2.3431),
    "팡테옹": (48.8463, 2.3460),
    "룩셈부르크 정원": (48.8462, 2.3371),
    "오르세 미술관": (48.8600, 2.3257),
    "샹젤리제 거리": (48.8696, 2.3079),
    "레 장발리드": (48.8565, 2.3126),
    "콩코르드 광장": (48.8656, 2.3211),
    "퐁피두 센터": (48.8606, 2.3522),
    "라 데팡스": (48.8900, 2.2400),
    "갤러리 라파예트": (48.8738, 2.3320),
    "그레뱅 박물관": (48.8718, 2.3422)
}

# 파리 경계 데이터 로드
with open('./파리_data/paris.geojson', 'r') as f:
    paris_boundary = json.load(f)

# shapely geometry 객체 생성
paris_area = shape(paris_boundary['features'][0]['geometry'])

# 마커 클러스터 생성
marker_cluster = MarkerCluster().add_to(map_paris)

# 마커 추가 조건 검사
def add_marker_if_inside(lat, lng, popup, icon_color):
    point = Point(lng, lat)
    if paris_area.contains(point):
        folium.Marker(
            location=[lat, lng],
            icon=folium.Icon(color=icon_color, icon='info-sign'),
            popup=popup
        ).add_to(marker_cluster)

# 사용자 데이터를 반복하면서 마커 추가
for idx, row in df_guest_prefer.iterrows():
    add_marker_if_inside(row['위도'], row['경도'], 'Guest Prefer', 'green')

for idx, row in df_non_guest_prefer.iterrows():
    add_marker_if_inside(row['위도'], row['경도'], 'Non-Guest Prefer', 'red')
    
# 각 랜드마크에 대한 마커 추가
for landmark, (lat, lng) in landmarks.items():
    folium.Marker(
        location=[lat, lng],
        icon=folium.Icon(icon='star', color='blue'),
        popup=landmark
    ).add_to(map_paris)


# df_guest_prefer의 위치 표시 (클러스터에 추가)
for idx, row in df_guest_prefer.iterrows():
    folium.Marker(
        location=[row['위도'], row['경도']],
        icon=folium.Icon(color='green', icon='info-sign'),
        popup='Guest Prefer'
    ).add_to(marker_cluster)

# df_non_guest_prefer의 위치 표시 (클러스터에 추가)
for idx, row in df_non_guest_prefer.iterrows():
    folium.Marker(
        location=[row['위도'], row['경도']],
        icon=folium.Icon(color='red', icon='info-sign'),
        popup='Non-Guest Prefer'
    ).add_to(marker_cluster)

# 지하철, 기차, 트램 데이터를 각각의 레이어로 추가
geojson_data_subway = './파리_data/paris_subway.geojson'
geojson_data_train = './파리_data/paris_train.geojson'
geojson_data_tram = './파리_data/paris_tram.geojson'

paris_boundary_geojson = './파리_data/paris.geojson'

# 파리 경계 레이어 추가
GeoJson(
    paris_boundary_geojson,
    name='Paris Boundary',
    style_function=lambda x: {
        'fillColor': 'blue',
        'color': 'blue',
        'weight': 2,
        'fillOpacity': 0.1
    }
).add_to(map_paris)

# 지하철 레이어
subway_layer = FeatureGroup(name='Subway Stations')
GeoJson(geojson_data_subway, name='Subway Stations').add_to(subway_layer)
subway_layer.add_to(map_paris)

# 기차 레이어
train_layer = FeatureGroup(name='Train Stations')
GeoJson(geojson_data_train, name='Train Stations').add_to(train_layer)
train_layer.add_to(map_paris)

# 트램 레이어
tram_layer = FeatureGroup(name='Tram Stops')
GeoJson(geojson_data_tram, name='Tram Stops').add_to(tram_layer)
tram_layer.add_to(map_paris)

# 레이어 컨트롤 추가
LayerControl().add_to(map_paris)

# 지도를 HTML 파일로 저장
map_paris.save('paris_transport_map_check1.html')

In [33]:
df_metro.columns

Index(['Identifiant station', 'Nom de la station', 'Capacité de la station',
       '위도', '경도'],
      dtype='object')

In [36]:
len(df_metro['역이름'].unique())

1466

In [39]:
df_metro['역이름']

0           Porte de Saint-Ouen - Bessières
1                    Ramponeau - Belleville
2                  Saint-Cloud - Hippodrome
3                   Porte Pouchet-Bessières
4       Gare du Nord - Hôpital Lariboisière
                       ...                 
1466               Place du Moulin de Javel
1467                     Belfort - Voltaire
1468                   Montreuil - Voltaire
1469                  Saint-Antoine Sévigné
1470            Gare de la Garenne-Colombes
Name: 역이름, Length: 1471, dtype: object

In [40]:
# 역 이름을 '-'로 분할하고 첫 번째 부분을 새로운 열에 저장
df_metro['역 이름 앞부분'] = df_metro['역이름'].apply(lambda x: x.split(' - ')[0])

# 앞부분이 같은 역 이름 찾기
common_stations = df_metro[df_metro.duplicated(subset=['역 이름 앞부분'], keep=False)]
print(common_stations)

       역_id                                  역이름  역수용인원         위도        경도  \
0     17044      Porte de Saint-Ouen - Bessières     42  48.897922  2.328515   
4     10107  Gare du Nord - Hôpital Lariboisière     49  48.881949  2.352339   
5     16039                Exelmans - Versailles     49  48.840975  2.264390   
6     11002                    Roquette - Thiéré     23  48.855079  2.373369   
10    11105              Parmentier - République     50  48.864495  2.375012   
...     ...                                  ...    ...        ...       ...   
1460  20030         Ménilmontant - Père Lachaise     41  48.863287  2.386994   
1461  22302         Gare RER - Général de Gaulle     41  48.933548  2.306473   
1464  22018      Charles de Gaulle - Les Sablons      0  48.880988  2.271575   
1467  11020                   Belfort - Voltaire     39  48.856336  2.382990   
1468  11012                 Montreuil - Voltaire     34  48.850507  2.393076   

                 역 이름 앞부분  
0     Porte

In [42]:
len(common_stations)

451

In [45]:
df_metro[df_metro['역수용인원']==0]

,역_id,역이름,역수용인원,위도,경도,역 이름 앞부분
50,14134,Château - Place de Catalogne,0,48.836806,2.318529,Château
75,23802,Raymond Poincaré - Allée de la Marche,0,48.838634,2.172410,Raymond Poincaré
77,11031_relais,Jacquard - Ternaux,0,48.864367,2.373093,Jacquard
122,30007,Pablo Picasso - Maurice Thorez,0,48.906888,2.448933,Pablo Picasso
135,19203,Pailleron - Bouret,0,48.880517,2.376390,Pailleron
190,15062,Convention - Lourmel,0,48.842055,2.285991,Convention
262,21404,Jean Jaurès - Gare de Clamart,0,48.813220,2.271677,Jean Jaurès
346,13035,Choisy - Tolbiac,0,48.825912,2.360239,Choisy
486,19204,Rampal - Belleville,0,48.873115,2.379841,Rampal
504,23801,Place de la Gare,0,48.838514,2.186614,Place de la Gare


In [47]:
df_metro[df_metro['역 이름 앞부분']=='Jacquard']

,역_id,역이름,역수용인원,위도,경도,역 이름 앞부분
77,11031_relais,Jacquard - Ternaux,0,48.864367,2.373093,Jacquard
1428,11031,Jacquard - Ternaux,0,48.864660,2.373162,Jacquard


In [4]:
import numpy as np
import pandas as pd

df1 = pd.read_csv('./check1.csv')
df1['게스트 선호 숙소 수'].unique()

array([  0, 103,  30,  29, 188,   4,   1,  60,  51,  14,  16,  40, 109,
        18, 144,  91,  35,  33, 192,  56,  24,  39,  26,  37,   7,  48,
        10,  31,  28,   6, 161,  41,  12,  34,  53,  50, 206,  46, 125,
        54, 101,  45, 123,  43,  62,  23,   2,  11,  27,  97,  52, 175,
        68,  44,  71,  17,  25, 205, 151,  57,  22,  49, 120, 118,  32,
         8,  55,  84, 199,  66,  96,  67,  92,  72,  20,   3,  13, 111,
       141, 176,  15,  42,  36,  83, 108, 119, 156, 126, 131, 191,  70,
        98,  61,  21,  78,  80,  95, 146,  75, 182,  79, 136, 106, 100,
        59, 158, 181, 121,   9, 112, 127,  58,  86, 211,  63, 142,  90,
        99,   5,  38,  47,  19,  77, 154,  82, 166,  69, 116, 157,  81,
       159,  89, 178, 138,  74, 124, 168,  73, 102, 117, 152, 114, 139,
        94, 185, 130, 194,  76,  88, 170, 128,  64, 132, 115, 134, 105,
       133, 122, 110, 148,  65,  85, 189, 197, 225, 174, 107, 212, 155,
       145], dtype=int64)

In [5]:
df1['게스트 선호 숙소 수'].sum()

42745

In [11]:
df1['비게스트 선호 숙소 수'].sum()

44221

In [8]:
df1

,역이름,게스트 선호 숙소 수,비게스트 선호 숙소 수
0,11 Novembre 1918,0,0
1,18 juin 1940,0,0
2,8 Mai 1945,0,0
3,Abbeville,103,106
4,Abbé Carton,30,38
...,...,...,...
1171,Wurtz,31,13
1172,Youri Gagarine,0,0
1173,Yves Farges,0,0
1174,Édouard Vaillant,0,0


In [12]:
len(df1)

1176

In [18]:
df1[df1['게스트 선호 숙소 수']>200]

,역이름,게스트 선호 숙소 수,비게스트 선호 숙소 수
65,Beaubourg,206,248
111,Bourg l'Abbé,205,295
409,Greneta,211,352
906,Porte Saint-Martin,205,316
1064,Square Emile,225,368
1130,Turbigo,212,327


In [19]:
df1[df1['게스트 선호 숙소 수']>100]

,역이름,게스트 선호 숙소 수,비게스트 선호 숙소 수
3,Abbeville,103,106
6,Aboukir,188,325
16,Alibert,109,89
20,Amelot,144,125
32,Archives,192,192
...,...,...,...
1130,Turbigo,212,327
1146,Victor Massé,152,113
1148,Vieille du Temple,155,122
1157,Vinaigriers,145,112
